## SArDeGnA (SpAtial Data Grid frAmework)
- SArDeGnA is a lightweight framework to manage spatial data in Python using a grid approach.
- This framework can help you to manage Point of Interests (i.e. pubs, restaurant) or points related to trajectories.


- In this notebook a I will explain how to use the spatial data framework.


## How to create a Grid
- A grid is made by cell.
- A grid is defined by specifying the coordinated (lat and lon) of the center and the radius  fo the grid and the radius of the cells.


In [1]:

from framework.grid import Grid

lng_center =  -0.06069 #this is defined in http://spatialreference.org/ref/epsg/wgs-84/
lat_center = 51.5437 # this is defined in http://spatialreference.org/ref/epsg/wgs-84/
radius_center = 1000 # this is defined in meter
radius_cell = 100 # this is defined in meter

grid = Grid()
grid.create_grid([lng_center, lat_center, radius_center], radius_cell)
# save the grid as txt file
grid.save_grid_as_txtfile('data/grid.txt')
# save information about indexes of the grid
grid.save_mapping_uni_bi_index('data/grid_as_uni_bi_index.txt')
# save information about indexe of the grid
grid.save_mapping_cell_index_lng_lat('data/grid_as_cell_index_lng_lat.txt')
#save grid as kml to use it in Google Earth
grid.save_grid_as_kmlfile('data/grid.kml')

In the file grid.txt, we have the following content (just some initial rows):
    * -0.0606900000@51.5437000000@1000@100
    * -0.0696643503@51.5381108271@0@0@0
    * -0.0678677197@51.5381108271@1@0@1
    * -0.0660710891@51.5381108271@2@0@2
    * -0.0642744586@51.5381108271@3@0@3
    * -0.0624778280@51.5381108271@4@0@4

- The first and second columns are the coordinate of the center (lng, lat) of the cells, the third is radius of the grid and the forth is the radius of the cell. 
- Then, for the other columns we have  the absolute index of the cell (named as unary index), and the relative index in terms of row and column of the grid (named as binary index).


In the case of grid_as_uni_bi_index.txt, we have the following content (just some initial rows):

    * 0:-0.0696643503@51.5381108271
    * 1:-0.0678677197@51.5381108271
    * 2:-0.0660710891@51.5381108271
    * 3:-0.0642744586@51.5381108271
    * 4:-0.0624778280@51.5381108271
    * 5:-0.0606811974@51.5381108271
    * 6:-0.0588845669@51.5381108271
    * 7:-0.0570879363@51.5381108271

- The first column is the absolute index of the cell an then you have the lng and lat of the center of the cell

In the case of grid_as_cell_index_lng_lat.txt, we have the following content (just some initial rows):

    * 0:0;0
    * 1:0;1
    * 2:0;2
    * 3:0;3
    * 4:0;4
    * 5:0;5
    * 6:0;6

The first column is the unary index and then you have the binary index.

In [2]:
#Example of use of The Spatial Reference System 4326
from osgeo import ogr, osr
spatialReference_4326 = osr.SpatialReference()
spatialReference_4326.ImportFromEPSGA(4326)
point = ogr.Geometry(ogr.wkbPoint)
point.AssignSpatialReference(spatialReference_4326)
point.AddPoint(-0.0696643503,51.5381108271)

In [3]:
print('This is the Lon',point.GetX())
print('This is the Lat',point.GetY())

This is the Lon -0.0696643503
This is the Lat 51.5381108271


In [4]:
#Example of use of The Spatial Reference System 3857
spatialReference_3857 = osr.SpatialReference()
spatialReference_3857.ImportFromEPSG(3857)

0

To get the list of cells, where each element has the coordinate of each corner plus the unary and binary index:

In [5]:
l_full = grid.get_list_cell_lng_lat_coord_as_bb_clock_wise()
l_center = grid.get_list_cell_lng_lat_center_index()

In [6]:
print(l_full[:4])

[[[(-0.06876603499934933, 51.538669570287446), (-0.06876603499934933, 51.537552076961965), (-0.07056266556758838, 51.537552076961965), (-0.07056266556758838, 51.538669570287446)], 0, [0, 0]], [[(-0.0669694044311103, 51.538669570287446), (-0.0669694044311103, 51.537552076961965), (-0.06876603499934933, 51.537552076961965), (-0.06876603499934933, 51.538669570287446)], 1, [0, 1]], [[(-0.06517277386287125, 51.538669570287446), (-0.06517277386287125, 51.537552076961965), (-0.0669694044311103, 51.537552076961965), (-0.0669694044311103, 51.538669570287446)], 2, [0, 2]], [[(-0.0633761432946322, 51.538669570287446), (-0.0633761432946322, 51.537552076961965), (-0.06517277386287125, 51.537552076961965), (-0.06517277386287125, 51.538669570287446)], 3, [0, 3]]]


Example of use of GEOJSON

In [7]:
from geojson import Feature, Point,FeatureCollection
list_geo_features = []
for (lng,lat),uni_index,*_ in l_center:
    geodata = Point((lng, lat))
    list_geo_features.append(Feature(geometry=geodata, id=uni_index, properties=[{'value': 'center_cell'}]))
geojson_data =FeatureCollection(list_geo_features)

Example of using ipyleaflet

In [8]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from ipyleaflet import *
center = [lat_center,lng_center]
zoom = 14
m = Map(center=center, zoom=zoom)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [9]:
g=GeoJSON(data=geojson_data)
m.add_layer(g)


In [10]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [11]:
row_dim,col_dim = grid.get_row_col_dim()
grid_list_by_row = [[[] for _ in range(col_dim)] for _ in range(row_dim)]
grid_list_by_col = [[[] for _ in range(row_dim)] for _ in range(col_dim)]

In [12]:
list_points_sw = []
for points,_,bin_index in l_full:
    grid_list_by_row[bin_index[0]][bin_index[1]]=points[0]
    grid_list_by_col[bin_index[1]][bin_index[0]]=points[0]

In [13]:


from geojson import LineString
from geojson import Feature, Point,FeatureCollection,LineString
list_geo_features_line_by_row = []
for index,row in enumerate(grid_list_by_row):
    geodata = LineString(row)
    list_geo_features_line_by_row.append(Feature(geometry=geodata, id=index))
list_geo_features_line_by_col = []
for index,col in enumerate(grid_list_by_col):
    geodata = LineString(col)
    list_geo_features_line_by_col.append(Feature(geometry=geodata, id=index+row_dim))
    
l_row_col = list_geo_features_line_by_col+list_geo_features_line_by_row 
geojson_data_grid =FeatureCollection(l_row_col)

Example of using Google Maps, but you need "your key"

In [14]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key="your key")

In [15]:
fig = gmaps.figure()
layer = gmaps.geojson_layer(geojson_data_grid, fill_color="green", stroke_color="green",fill_opacity=0.8)
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In the file geoutils, there are some functions such as 
    * serialize_list_point_lon_lat_index_as_kml
    * serialize_list_point_lon_lat_index_as_line_in_kml
That you can use to serialize the file in kml and open the result in [qgis](http://www.qgis.org/en/site/index.html) or [google earth](https://www.google.com/intl/en/earth/) 